In [3]:
# !pip install gym

  Using cached gym-0.26.2.tar.gz (721 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached cloudpickle-3.1.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached gym_notices-0.0.8-py3-none-any.whl.metadata (1.0 kB)
Using cached cloudpickle-3.1.1-py3-none-any.whl (20 kB)
Using cached gym_notices-0.0.8-py3-none-any.whl (3.0 kB)
  Created wheel for gym: filename=gym-0.26.2-py3-none-any.whl size=827704 sha256=509d600336856bae4f96e16f3369f823ae48f48d2ac5a9f82b05885fb74cceb7
  Stored in directory: c:\users\tejas\appdata\local\pip\cache\wheels\1d\34\c6\856a1e1eff47d8f18545c833b6138ae1e9f53c7de9bcc5f31d
Successfully built gym


In [5]:
import gym
import numpy as np
import random
from gym import spaces

In [ ]:
class MentalHealthEnv(gym.Env):
    def __init__(self, df):
        super(MentalHealthEnv, self).__init__()

        #loading the dataset
        self.df = df.copy()
        self.current_index = 0


        #define state: 7 mental health scores (ranging between 0 to 300)
        self.observation_space = spaces.Box(low = 0, high = 300, shape = (7,), dtype = np.float32)

        # defining action space
        self.actions = [
            "Engage in Outdoor activities",
            "Reduce the social media usage",
            "Watch educational videos",
            "Increase mindfulness practicess like yoga",
            "Interact with positive communities",
            "Avoid stress-inducing content",
            "Follow a structured daily routine"
        ]

        self.action_space = spaces.Discrete(len(self.actions))

        #initiating state: random values between 50 and 250 (to avoid zero-starting issues)
        self.state = np.random.uniform(low = 50, high = 250, size = (7,))

        # define action effects (each action affects specific scores)
        self.action_effects = {
            "Engage in Outdoor activities": np.array([10, 5, 0, 8, -5, 7, 0]),
            "Reduce the social media usage": np.array([5, -15, 0, 0, 10, 0, 0]),
            "Watch educational videos": np.array([5, 0, 15, 7, -3, 10, 0]),
            "Increase mindfulness practicess like yoga": np.array([12, 0, 0, 5, -8, 15, 0]),
            "Interact with positive communities": np.array([6, 0, -8, 0, 12, 0, 0]),
            "Avoid stress-inducing content": np.array([6, 0, -8, 0, 12, 0, 0]),
            "Follow a structured daily routine": np.array([10, 0, 0, 0, 7, 8, 0])
        }
        self.state = self.get_state_from_df()


    def get_state_from_df(self):
        # extract the initial state from the dataframe based on the current index
        row = self.df.iloc[self.current_index]
        return np.array(row[:-1])
    
    def step(self, action):
        #apply an action and update mental health scores

        self.state = np.clip(self.state)





